In [1]:
import face_recognition
import cv2
from datetime import datetime, timedelta
import numpy as np
import platform
import pickle

from PIL import Image
import PIL
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from skimage import transform

In [2]:
# Загружаем обученную модель
model_a = keras.models.load_model("model_a_regression")

In [4]:
def main_loop():
    
    video_capture = cv2.VideoCapture(0) # захватываем видео с камеры

    while True:

        ret, frame = video_capture.read() # вытаскиваем изображение

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25) # уменьшаем, чтоб легче обрабатывать

        rgb_small_frame = small_frame[:, :, ::-1] # меняем bgr на rgb

        face_locations = face_recognition.face_locations(rgb_small_frame) # находим лица, если они есть в видео

       
        for (top, right, bottom, left) in face_locations:
            
            face_image = rgb_small_frame[top:bottom, left:right] # вытаскиваем изображение лица по координатам
            face1 = face_image/255. # нормализуем изображение
            face1 = transform.resize(face1, (100, 100, 3)) # ресайзим в целевой размер
            face1 = np.expand_dims(face1, axis=0) # добавляем дополнительное измерение

            age = model_a.predict(face1)[0][0].round(0) # предсказываем возраст

            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
           
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            cv2.putText(frame, f'Age: {age}', (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
        
        cv2.imshow('Video', frame)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


   
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    main_loop()